### XGBoost for Time Series Forecasting

Using XGBoost (one of the variant of Boosting) for forecasting stock price. [XGBoost documentation](https://xgboost.readthedocs.io/en/latest/)
- XGBoost is an ensemble of decision trees where new trees fix errors of the trees that are already part of the model. It is therefore, trees are added until no further improvements can be made to the model.
- In order to use XGBoost for time series, we need to evaluate the model via `walk-forward validation` instead of `k-fold cross validation` because k-fold sometimes would have biased results.

#### Importing Extensions and Libraries

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext watermark
%load_ext lab_black

In [2]:
import os
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

np.random.seed(22)

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

# make autocomplet working
%config Completer.use_jedi = False

In [3]:
%watermark -iv -v

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.22.0

numpy     : 1.20.2
pandas    : 1.2.3
matplotlib: 3.4.1



#### Loading dataset from yahoo finance 

In [4]:
# data is microsoft's past 5 year daily stock price (April 10 2020 - April 10 2021)
data_url = "https://query1.finance.yahoo.com/v7/finance/download/MSFT?period1=1460246400&period2=1618012800&interval=1d&events=history&includeAdjustedClose=true"
data = pd.read_csv(data_url)
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-04-11,54.490002,55.150002,54.299999,54.310001,49.800735,21414200
1,2016-04-12,54.369999,54.779999,53.759998,54.650002,50.112518,24944300
2,2016-04-13,55.119999,55.439999,54.889999,55.349998,50.754379,20818000
3,2016-04-14,55.220001,55.580002,55.070000,55.360001,50.763554,20877100
4,2016-04-15,55.299999,55.919998,55.110001,55.650002,51.029476,28793800
...,...,...,...,...,...,...,...
1254,2021-04-05,242.759995,249.960007,242.699997,249.070007,249.070007,36910600
1255,2021-04-06,247.610001,249.399994,246.880005,247.860001,247.860001,22931900
1256,2021-04-07,247.809998,250.929993,247.190002,249.899994,249.899994,22719800
1257,2021-04-08,252.770004,254.139999,252.000000,253.250000,253.250000,23625200


In [5]:
# for simplicity, I am just taking the closing price in this forecast (making it a univariate time series problem)
df = data[["Close"]].round(2).copy()
print(df.shape)
df.head()

(1259, 1)


,Close
0,54.31
1,54.65
2,55.35
3,55.36
4,55.65


#### Transforming the univariate problem to a supervised problem
For this particular problem, I am going to take the target value as the next day's stock price and drop the NaN of the last row in target column because for the last day, we don't have any shift, meaning, we don't have next days value.

In [6]:
# adding target column
df["target"] = df.Close.shift(-1)
df

,Close,target
0,54.31,54.65
1,54.65,55.35
2,55.35,55.36
3,55.36,55.65
4,55.65,56.46
...,...,...
1254,249.07,247.86
1255,247.86,249.90
1256,249.90,253.25
1257,253.25,255.85


In [7]:
# drop the NaN values in the last row in target column
df.dropna(inplace=True)
print(df.shape)
df.head()

(1258, 2)


,Close,target
0,54.31,54.65
1,54.65,55.35
2,55.35,55.36
3,55.36,55.65
4,55.65,56.46


#### Splitting the dataset into train and test dataset
Generally we split data as dependent and independent but here I am splitting the dataset which includes both values.

In [8]:
# custom function for train test split
def train_test_split(data, perc):
    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

In [9]:
# 80-20 split of data into train and test respectively
train, test = train_test_split(df, 0.2)

In [10]:
print(len(df))
print(len(train))
print(len(test))

1258
1006
252


In [11]:
df.shape, train.shape, test.shape

((1258, 2), (1006, 2), (252, 2))

#### Training with XGBRegressor
I am going to use XGBRegressor model to train and predict based on that. It is the implementation of the scikit-learn API for XGBoost regression.

In [12]:
# lets split the train as dependent and independent to fit into the model
X_train = train[:, :-1]
y_train = train[:, -1]

In [13]:
X_train.shape, y_train.shape

((1006, 1), (1006,))

In [14]:
# XGBRegressor??

In [15]:
%%time
# using 100 trees and all cpu cores
model = XGBRegressor(n_estimators=1000, n_jobs=-1)
model.fit(X_train, y_train)

CPU times: user 6.12 s, sys: 0 ns, total: 6.12 s
Wall time: 1.07 s


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=-1, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

#### Prediction

In [16]:
# lets take one of the test data
test[0]

array([165.13, 165.14])

In [17]:
# we need to pass the first array and the model should predict the second array as we have our dataset maintained in this way. lets extract that as variable val.
val = np.array(test[0, 0]).reshape(1, -1)
val

array([[165.13]])

In [18]:
# now, lets predict the next day's stock based on the val value. the model must predict 224.97
pred = model.predict(val)
pred[0]

162.29034

Its not close as the value differs almost 3 dollars.

**Now the next step would be to use Walk-forward validation to make the prediction more correct**

#### Walk-forward Validation
- In timeseries, as we make one step forward prediction, in our case one day prediciton, we will predict the first record in the test dataset first
- After predicting the first row of test dataset, we add real observation from the test dataset to the train dataset, refit the model and predict the next one in the test dataset.
- RMSE metric from scikit-learn is used for evaluation

In [19]:
# lets create a function for the above prediction to predict one sample at a time
def xgb_predict(train, val):
    train = np.array(train)
    X_train, y_train = train[:, :-1], train[:, -1]
    model = XGBRegressor(n_estimators=1000, n_jobs=1)
    model.fit(X_train, y_train)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

In [20]:
# now, lets create a function for walk-forward validation
def validate(dataset, perc):
    predictions = []
    train, test = train_test_split(dataset, perc)
    history = [x for x in train]

    for i in range(len(test)):
        X_test, y_test = test[i, :-1], test[i, -1]

        pred = xgb_predict(history, X_test[0])
        predictions.append(pred)
        history.append(test[i])

    error = mean_squared_error(test[:, -1], predictions, squared=False)

    return error, test[:, -1], predictions

In [21]:
%%time
rmse, y, pred = validate(df, 0.2)

CPU times: user 2min 37s, sys: 0 ns, total: 2min 37s
Wall time: 2min 37s


In [22]:
rmse

5.755341290843571

In [23]:
y

array([165.14, 165.51, 173.7 , 171.88, 177.04, 178.6 , 175.06, 167.82,
       173.52, 171.42, 174.55, 174.05, 169.81, 177.43, 179.21, 174.57,
       178.84, 180.76, 182.54, 183.6 , 184.68, 186.74, 182.51, 179.75,
       180.53, 183.16, 184.91, 183.63, 185.66, 183.43, 183.51, 181.57,
       181.81, 181.4 , 183.25, 182.83, 184.91, 185.36, 182.92, 187.2 ,
       188.36, 189.8 , 196.84, 186.27, 187.74, 188.94, 193.57, 194.24,
       196.32, 195.15, 200.57, 201.91, 197.84, 200.34, 196.33, 198.44,
       203.51, 204.7 , 206.26, 210.7 , 208.25, 212.83, 214.32, 213.67,
       207.07, 208.35, 208.04, 203.92, 202.88, 211.6 , 208.75, 211.75,
       202.54, 201.3 , 203.85, 202.02, 204.06, 203.9 , 205.01, 216.54,
       213.29, 212.94, 216.35, 212.48, 208.25, 203.38, 209.19, 208.7 ,
       208.9 , 210.28, 211.49, 209.7 , 214.58, 213.02, 213.69, 216.47,
       221.15, 226.58, 228.91, 225.53, 227.27, 231.65, 217.3 , 214.25,
       202.66, 211.29, 205.37, 204.03, 205.41, 208.78, 205.05, 202.91,
      

In [27]:
# lets print as array to make the list not go long in notebook
np.array(pred)

array([162.29034, 165.13068, 168.02562, 180.11336, 170.21933, 170.8971 ,
       170.89839, 180.10664, 172.76816, 171.87741, 177.02258, 180.10957,
       180.10559, 158.18912, 178.5929 , 175.05698, 174.04736, 175.05089,
       179.90991, 183.89334, 183.89748, 183.71375, 187.27536, 183.5983 ,
       183.62088, 182.53027, 184.67363, 183.71602, 183.8991 , 187.22154,
       184.6882 , 183.5247 , 182.53796, 181.80783, 181.80844, 184.89937,
       183.60004, 183.6379 , 187.22168, 184.90533, 187.26846, 184.44022,
       184.44356, 196.8357 , 182.5174 , 184.43224, 184.44499, 186.27078,
       194.24089, 186.27464, 196.31168, 186.27466, 201.905  , 186.27919,
       201.90355, 195.15099, 200.33235, 197.84314, 204.69504, 206.25691,
       210.69498, 210.69467, 208.2522 , 214.31546, 213.67407, 210.69902,
       212.82364, 212.82166, 204.69786, 204.68613, 208.25047, 208.04907,
       208.74904, 211.5864 , 197.85661, 202.89143, 197.85518, 202.8797 ,
       202.88177, 206.26154, 213.66449, 207.08072, 

#### Conclusion
- The `rmse` value is not that bad but we can improve if needed by using better models.
- I just predicted the model with using `close price`but in real scenario, people generally use all other columns shown in the dataset and also other sources that can add value to the model as stock market is complex and needs many useful considerations.
- In real life scenario, this model is not suitable for for testing, it gives a overall idea how to do timeseries forecasting using XGBoost.